<a href="https://colab.research.google.com/github/Ilaharshith/Reinforcement-Learning-/blob/main/LAB_08.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import math
import time
import random
from collections import namedtuple
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Gymnasium is the modern replacement for Gym
try:
    import gymnasium as gym
except ImportError:
    raise ImportError("Please install gymnasium: pip install gymnasium[classic-control]")


# -------------------- Actor Network --------------------
class Actor(nn.Module):
    def __init__(self, obs_dim, act_dim, hidden_sizes=(64, 64), log_std_init=-0.5):
        super().__init__()
        layers = []
        last = obs_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        self.net = nn.Sequential(*layers)
        self.mean = nn.Linear(last, act_dim)
        self.log_std = nn.Parameter(torch.ones(act_dim) * log_std_init)

    def forward(self, obs):
        x = self.net(obs)
        mean = self.mean(x)
        log_std = self.log_std.expand_as(mean)
        std = torch.exp(log_std)
        return mean, std, log_std


# -------------------- Critic Network --------------------
class Critic(nn.Module):
    def __init__(self, obs_dim, hidden_sizes=(64, 64)):
        super().__init__()
        layers = []
        last = obs_dim
        for h in hidden_sizes:
            layers.append(nn.Linear(last, h))
            layers.append(nn.ReLU())
            last = h
        layers.append(nn.Linear(last, 1))
        self.net = nn.Sequential(*layers)

    def forward(self, obs):
        return self.net(obs).squeeze(-1)


# -------------------- Utilities --------------------
Transition = namedtuple('Transition', ['obs', 'act', 'rew', 'next_obs', 'done', 'logp'])

def gaussian_log_prob(mean, log_std, act):
    var = torch.exp(2 * log_std)
    logp = -0.5 * (((act - mean) ** 2) / var + 2 * log_std + math.log(2 * math.pi))
    return logp.sum(axis=-1)

def sample_action(mean, std):
    eps = torch.randn_like(mean)
    return mean + eps * std


# -------------------- A2C Agent --------------------
class A2CAgent:
    def __init__(self, obs_dim, act_dim, device='cpu', actor_lr=3e-4, critic_lr=1e-3,
                 gamma=0.99, value_coef=0.5, entropy_coef=1e-3, max_grad_norm=0.5):
        self.device = device
        self.actor = Actor(obs_dim, act_dim).to(device)
        self.critic = Critic(obs_dim).to(device)
        self.optimizer_actor = optim.Adam(self.actor.parameters(), lr=actor_lr)
        self.optimizer_critic = optim.Adam(self.critic.parameters(), lr=critic_lr)
        self.gamma = gamma
        self.value_coef = value_coef
        self.entropy_coef = entropy_coef
        self.max_grad_norm = max_grad_norm

    def get_action(self, obs):
        obs_t = torch.as_tensor(obs, dtype=torch.float32, device=self.device).unsqueeze(0)
        with torch.no_grad():
            mean, std, log_std = self.actor(obs_t)
            act = sample_action(mean, std)
            logp = gaussian_log_prob(mean, log_std, act)
            value = self.critic(obs_t)
        return act.cpu().numpy()[0], logp.cpu().numpy()[0], value.cpu().numpy()[0]

    def update(self, transitions, last_value=0.0):
        obs = torch.as_tensor(np.vstack([t.obs for t in transitions]), dtype=torch.float32, device=self.device)
        acts = torch.as_tensor(np.vstack([t.act for t in transitions]), dtype=torch.float32, device=self.device)
        rewards = [t.rew for t in transitions]
        dones = [t.done for t in transitions]
        old_logps = torch.as_tensor(np.array([t.logp for t in transitions]), dtype=torch.float32, device=self.device)

        # Compute returns (bootstrapped)
        returns = []
        R = last_value
        for r, done in zip(reversed(rewards), reversed(dones)):
            R = r + self.gamma * R * (1.0 - float(done))
            returns.insert(0, R)
        returns = torch.as_tensor(returns, dtype=torch.float32, device=self.device)

        # Value and policy losses
        values = self.critic(obs)
        mean, std, log_std = self.actor(obs)
        new_logps = gaussian_log_prob(mean, log_std, acts)
        entropy = (0.5 * (1.0 + math.log(2 * math.pi)) + log_std).sum(-1).mean()

        advantages = returns - values.detach()
        policy_loss = -(new_logps * advantages).mean()
        value_loss = 0.5 * (returns - values).pow(2).mean()
        actor_loss = policy_loss - self.entropy_coef * entropy

        # Update actor
        self.optimizer_actor.zero_grad()
        actor_loss.backward()
        nn.utils.clip_grad_norm_(self.actor.parameters(), self.max_grad_norm)
        self.optimizer_actor.step()

        # Update critic
        self.optimizer_critic.zero_grad()
        value_loss.backward()
        nn.utils.clip_grad_norm_(self.critic.parameters(), self.max_grad_norm)
        self.optimizer_critic.step()

        return dict(policy_loss=policy_loss.item(), value_loss=value_loss.item(), entropy=entropy.item())


# -------------------- Training Loop --------------------
def train(env_name='Pendulum-v1', seed=1, episodes=500, max_steps=200, n_steps=5, render=False):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    env = gym.make(env_name)
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)

    obs_dim = env.observation_space.shape[0]
    act_dim = env.action_space.shape[0]
    agent = A2CAgent(obs_dim, act_dim, device=device)

    total_rewards = []
    for ep in range(1, episodes + 1):
        obs, info = env.reset(seed=seed)
        ep_reward = 0.0
        done = False
        step = 0
        buffer = []

        while not done and step < max_steps:
            act, logp, value = agent.get_action(obs)
            next_obs, rew, terminated, truncated, info = env.step(act)
            done_flag = terminated or truncated

            buffer.append(Transition(obs=obs, act=act, rew=rew, next_obs=next_obs, done=done_flag, logp=logp))
            ep_reward += rew
            obs = next_obs
            step += 1

            if len(buffer) >= n_steps or done_flag or step >= max_steps:
                if done_flag:
                    last_value = 0.0
                else:
                    obs_t = torch.as_tensor(obs, dtype=torch.float32, device=device).unsqueeze(0)
                    with torch.no_grad():
                        last_value = agent.critic(obs_t).cpu().numpy()[0]
                stats = agent.update(buffer, last_value=last_value)
                buffer = []

            if done_flag:
                break

        total_rewards.append(ep_reward)
        if ep % 10 == 0:
            avg = np.mean(total_rewards[-50:])
            print(f"Ep {ep}\tAvgReward(50)={avg:.2f}\tLastReward={ep_reward:.2f}\t"
                  f"PolicyLoss={stats['policy_loss']:.4f}\tValueLoss={stats['value_loss']:.4f}\tEntropy={stats['entropy']:.4f}")

    env.close()
    return agent, total_rewards


# -------------------- Entry Point --------------------
if __name__ == "__main__":
    start = time.time()
    agent, rewards = train(episodes=60, max_steps=200, n_steps=5)
    print('Training finished in', round(time.time() - start, 2), 'seconds.')


Ep 10	AvgReward(50)=-1027.73	LastReward=-1200.86	PolicyLoss=29.9340	ValueLoss=619.6361	Entropy=0.9358
Ep 20	AvgReward(50)=-1231.11	LastReward=-1564.66	PolicyLoss=278.6509	ValueLoss=31455.6992	Entropy=0.9618
Ep 30	AvgReward(50)=-1334.03	LastReward=-1462.08	PolicyLoss=515.6377	ValueLoss=298174.7500	Entropy=0.9853
Ep 40	AvgReward(50)=-1306.81	LastReward=-1398.27	PolicyLoss=684.8461	ValueLoss=277495.1875	Entropy=0.9840
Ep 50	AvgReward(50)=-1309.60	LastReward=-1345.48	PolicyLoss=803.1285	ValueLoss=218649.7188	Entropy=0.9761
Ep 60	AvgReward(50)=-1328.08	LastReward=-1117.27	PolicyLoss=714.1961	ValueLoss=696091.0000	Entropy=0.9751
Training finished in 25.61 seconds.
